## Vision por Computador: Práctica 6 

# Propuesta 1: Juego razas

### Importaciones necesarias

In [13]:
# Importa los modulos necesarios
import cv2
import os
from deepface import DeepFace

### Clases para gestionar la interfaz

In [14]:
class DynamicButton:
    def __init__(self, x1, y1, width, height, res_width, res_height):
        # Calcular las proporciones en función de la resolución de referencia (900x699)
        ref_width, ref_height = 900, 699
        self.width = width * res_width / ref_width
        self.height = height * res_height / ref_height
        self.x1 = x1 * res_width / ref_width
        self.y1 = y1 * res_height / ref_height

    def get_coordinates(self):
        x2 = self.x1 + self.width
        y2 = self.y1 + self.height
        return (int(self.x1), int(self.y1), int(x2), int(y2))

class DynamicButtons:
    def __init__(self, res_width, res_height):
        self.MENU = {
            "ELEGIR": DynamicButton(50, 285, 245, 105, res_width, res_height).get_coordinates(),
            "DETECTAR": DynamicButton(325, 285, 245, 105, res_width, res_height).get_coordinates(),
            "SALIR": DynamicButton(600, 285, 245, 105, res_width, res_height).get_coordinates(),
        }

        self.DETECTAR = {
            "MENU": DynamicButton(35, 580, 160, 80, res_width, res_height).get_coordinates(),
            "SALIR": DynamicButton(710, 580, 160, 80, res_width, res_height).get_coordinates()
        }

        self.ELEGIR = {
            "MENU": DynamicButton(35, 15, 160, 80, res_width, res_height).get_coordinates(),
            "SALIR": DynamicButton(710, 15, 160, 80, res_width, res_height).get_coordinates()
        }

        self.CAMBIAR_SOMBRERO = {
            'black': DynamicButton(55, 545, 110, 110, res_width, res_height).get_coordinates(),
            'indian': DynamicButton(190, 545, 110, 110, res_width, res_height).get_coordinates(),
            'asian': DynamicButton(325, 545, 110, 110, res_width, res_height).get_coordinates(),
            'latino hispanic': DynamicButton(460, 545, 110, 110, res_width, res_height).get_coordinates(),
            'middle eastern': DynamicButton(595, 545, 110, 110, res_width, res_height).get_coordinates(),
            'white': DynamicButton(735, 545, 110, 110, res_width, res_height).get_coordinates(),
        }

# ------------ EJEMPLO DE USO ------------ 
# dynamic_buttons = DynamicButtons(width, height)  
# print(dynamic_buttons.botones_menu)


class GameInterface:
    def __init__(self, res_width, res_height):
        self.res_width = res_width
        self.res_height = res_height
        self.MENU = self.cargar_y_redimensionar('assets/pantalla_menu.png')
        self.DETECTAR = self.cargar_y_redimensionar('assets/pantalla_detectar.png')
        self.ELEGIR = {
            'white': self.cargar_y_redimensionar('assets/GorroBlancoSeleccionado.png'),
            'indian': self.cargar_y_redimensionar('assets/GorroIndioSeleccionado.png'),
            'asian': self.cargar_y_redimensionar('assets/GorroChinoSeleccionado.png'),
            'latino hispanic': self.cargar_y_redimensionar('assets/GorroMexicanoSeleccionado.png'),
            'black': self.cargar_y_redimensionar('assets/GorroNegroSeleccionado.png'),
            'middle eastern': self.cargar_y_redimensionar('assets/GorroRusoSeleccionado.png')
        }

    def cargar_y_redimensionar(self, ruta_imagen):
        imagen = cv2.imread(ruta_imagen, cv2.IMREAD_UNCHANGED)
        imagen_redimensionada = cv2.resize(imagen, (self.res_width, self.res_height))
        return imagen_redimensionada
    
    @staticmethod
    def add_interface(frame, interface):
        """Superpone una interfaz gráfica en el frame."""
        # interface = cv2.resize(interface, (frame.shape[1], frame.shape[0]))
        alpha_s = interface[:, :, 3] / 255.0
        alpha_l = 1 - alpha_s

        for c in range(0, 3):
            frame[:, :, c] = (alpha_s * interface[:, :, c] +
                              alpha_l * frame[:, :, c])
        return frame
    
# ------------  Ejemplo de uso------------ 
# game_interface = GameInterface(1280, 720)

class GameState:
    def __init__(self):
        self.current_state = "MENU" # Estados posibles: "MENU", "DETECTAR", "ELEGIR", "SALIR"
        self.selected_hat = 'white'

    def change_state(self, new_state):
        self.current_state = new_state

    def set_selected_hat(self, hat):
        self.selected_hat = hat

# ------------  Ejemplo de uso------------ 
# game_state = GameState()
# game_state.change_state("DETECTAR")



### Clases para gestionar los sombreros

In [15]:
class HatManager:
    def __init__(self):
        self.hats = {
            'middle eastern': self.load_hat('assets/gorro_ruso.png'),
            'asian': self.load_hat('assets/gorro_chino.png'),
            'indian': self.load_hat('assets/gorro_indio.png'),
            'latino hispanic': self.load_hat('assets/gorro_mexicano.png'),
            'black': self.load_hat('assets/gorro_negro.png'),
            'white': self.load_hat('assets/gorro_blanco.png'),
            'default': self.load_hat('assets/gorro_blanco.png')
        }

    def load_hat(self, path):
        return cv2.imread(path, cv2.IMREAD_UNCHANGED)

    def get_hat(self, hat_name):
        return self.hats.get(hat_name, self.hats['default'])
    
    def show_available_hats(self):
        for nombre, gorro in self.hats.items():
            print(f"{nombre}: {gorro.shape}")  # Debería mostrar algo como (altura, anchura, 4)


class HatOverlayManager:
    def __init__(self, hat_manager, min_confidence=2, analysis_frequency=2):
        self.hat_manager = hat_manager
        self.counter_frames = 0
        self.last_analysis = None
        self.min_confidence = min_confidence
        self.analysis_frequency = analysis_frequency

    def analyze_and_overlay(self, frame):
        """Analiza las caras y superpone gorros."""
        self.counter_frames += 1
        if self.counter_frames % self.analysis_frequency == 0:
            self.last_analysis = DeepFace.analyze(img_path=frame, enforce_detection=False, actions=['race'])

        if self.last_analysis:
            for result in self.last_analysis:
                if result['face_confidence'] >= self.min_confidence:
                    # Extraer la región donde se encuentra el rostro y la raza dominante
                    region, dominant_race = result['region'], result['dominant_race']
                    # Obtiene las coordenadas de la cara
                    x, y, w, h = region['x'], region['y'], region['w'], region['h']
                    
                    # Superponer el gorro en la cara detectada
                    hat = self.hat_manager.get_hat(dominant_race)
                    frame = self.overlay_hat(frame, hat, x, y, w, h)
                    # Opcional: Dibujar rectángulo y texto
                    # cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    # font, fontScale, color, thickness, pos = cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, (50, 50)
                    # cv2.putText(frame, raza_dominante, (x, y - 10), font, fontScale, color, thickness, cv2.LINE_AA)
        return frame

    @staticmethod
    def overlay_hat(frame, hat, x, y, w, h):
        """Superpone un gorro en un frame"""
        resized_hat = cv2.resize(hat, (w, int(h / 2)))
        for i in range(resized_hat.shape[0]):
            for j in range(resized_hat.shape[1]):
                if resized_hat[i, j, 3] != 0:
                    frame[y - int(h / 2) + i, x + j, :] = resized_hat[i, j, 0:3]
        return frame

    def choose_and_overlay(self, frame, selected_hat):
        """Toma un gorro elegido y lo superpone en la imagen"""
        # Define el clasificador de rostros
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        
        # Detecta las caras en la imagen
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)        
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=6, minSize=(150, 150))
        
        # Superponer el gorro seleccionado sobre cada cara detectada
        for (x, y, w, h) in faces:
            hat = self.hat_manager.get_hat(selected_hat)
            frame = self.overlay_hat(frame, hat, x, y, w, h)
            # cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        return frame




In [16]:
class Game:
    def __init__(self, source_video, game_interface=None, dynamic_buttons=None, overlay_manager=None):
        self.vid = self.__check_source(source_video)
        self.game_state = GameState()
        self.game_interface = game_interface if game_interface is not None else self.__init_component(GameInterface)
        self.dynamic_buttons = dynamic_buttons if dynamic_buttons is not None else  self.__init_component(DynamicButtons)
        self.overlay_manayer = overlay_manager if overlay_manager is not None else HatOverlayManager(hat_manager=HatManager(),
                                                                                                     min_confidence = 3,analysis_frequency=2 )

        
    def __check_source(self, source):
        vid = cv2.VideoCapture(source)
        if not vid.isOpened():
            raise Exception(f"Error al abrir la fuente de video: {source}")
        return vid
    
    def __init_component(self, ComponentClass):
        width = int(self.vid.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(self.vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
        return ComponentClass(res_height=height, res_width=width)
        
    def start(self):
        
        cv2.namedWindow('Juego')
        cv2.setMouseCallback('Juego', self.clic_mouse, {'dynamic_buttons': self.dynamic_buttons})

        while True:
            ret, frame = self.vid.read()
            if not ret:
                print("No se pudo leer el frame. Fin del video o error de lectura.")
                break

            self.manage_game_state(frame)

            cv2.imshow('Juego', frame)
            
            if (cv2.waitKey(1) & 0xFF == 27) or self.game_state.current_state == "SALIR":
                break

        self.vid.release()
        cv2.destroyAllWindows()

    def exit(self):
        self.game_state.current_state = "SALIR"   
        
    def manage_game_state(self, frame):
        estado = self.game_state.current_state
        if estado == "MENU":
            self.integrate_interface(frame, self.game_interface.MENU, self.dynamic_buttons.MENU)
        elif estado == "DETECTAR":
            frame = self.overlay_manayer.analyze_and_overlay(frame) #  detectar(frame)
            self.integrate_interface(frame, self.game_interface.DETECTAR, self.dynamic_buttons.DETECTAR)
        elif estado == "ELEGIR":
            frame = self.overlay_manayer.choose_and_overlay(frame, self.game_state.selected_hat) # elegir(frame, self.game_state.selected_hat)
            self.integrate_interface(frame, self.game_interface.ELEGIR[self.game_state.selected_hat], self.dynamic_buttons.ELEGIR)
       
        # No se necesita un caso para "SALIR", ya que se maneja en el bucle principal

    def integrate_interface(self, frame, pantalla, botones):
        self.game_interface.add_interface(frame=frame, interface=pantalla)
        #for nombre_boton, (x1, y1, x2, y2) in botones.items():
        #    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    def clic_mouse(self, event, x, y, flags, param):
        buttons = param['dynamic_buttons']
        if event == cv2.EVENT_LBUTTONDOWN:
            estado = self.game_state.current_state
            if estado == "MENU":
                self.update_state(x, y, buttons.MENU)
            elif estado == "DETECTAR":
                self.update_state(x, y, buttons.DETECTAR)
            elif estado == "ELEGIR":
                self.update_state(x, y, buttons.ELEGIR)
                self.update_selected_hat(x, y, buttons.CAMBIAR_SOMBRERO)

    def update_state(self, x, y, botones):
        for nombre_boton, (x1, y1, x2, y2) in botones.items():
            if x1 <= x <= x2 and y1 <= y <= y2:
                self.game_state.change_state(nombre_boton)
                break

    def update_selected_hat(self, x, y, botones):
        for nombre_gorro, (x1, y1, x2, y2) in botones.items():
            if x1 <= x <= x2 and y1 <= y <= y2:
                self.game_state.set_selected_hat(nombre_gorro)
                break
            
# Con overlay Manager personalizado
# ov_manager_2 = HatOverlayManager(hat_manager=HatManager(),min_confidence = 3,analysis_frequency=10)
# game = Game('./video_test_5.mp4', overlay_manager= ov_manager_2)
# game.start()

# Con overlay manager por defecto
game = Game(0)
game.start()


No se pudo leer el frame. Fin del video o error de lectura.
